In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from utils_new import extract_PID_data, resample_PID_data, Throttle_Constraint, RPM_Constraint, Speed_Constraint, Remove_Outliers
import os
import json
from operator import itemgetter
import seaborn as sns
#from scipy.stats import pearson
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
import plotly.graph_objects as go
from datetime import datetime, timezone
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy import signal
import plotly.express as px
import requests
import time

In [2]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [3]:
def RPM_Contraint(DATA,T1,RANGE):

    TEMP = DATA[0,:] # 0th is RPM
    nums = TEMP[(RANGE[0] < TEMP) & (TEMP <= RANGE[1])]
    Samples = len(nums)
    #print(Samples)
    DATA_OUT = np.zeros((DATA.shape[0],Samples))
    T_OUT = np.zeros(Samples)

    out_cnt = 0

    for cnt in range(0,DATA.shape[1]):
        if ((RANGE[0] < TEMP[cnt]) and (TEMP[cnt] <= RANGE[1])):
            for var_cnt in range(0,DATA.shape[0]):
                DATA_OUT[var_cnt,out_cnt] = DATA[var_cnt,cnt]
            T_OUT[out_cnt] = T1[cnt]
            out_cnt = out_cnt + 1

    return DATA_OUT,T_OUT
    

def Throttle_Contraint(DATA,T2,TH):

    TEMP = DATA[2,:] # 7th is Throttle
    nums = TEMP[(TH < TEMP)]
    Samples = len(nums)
    #print(Samples)
    DATA_OUT = np.zeros((DATA.shape[0],Samples))
    T_OUT = np.zeros(Samples)

    out_cnt = 0

    for cnt in range(0,DATA.shape[1]):
        if (TH < TEMP[cnt]):
            for var_cnt in range(0,DATA.shape[0]):
                DATA_OUT[var_cnt,out_cnt] = DATA[var_cnt,cnt]
            T_OUT[out_cnt] = T2[cnt]
            out_cnt = out_cnt + 1

    return DATA_OUT,T_OUT

In [4]:
def extract_PID_data_v3(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = '106'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '92'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '190'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '183'
        elif LABEL == 'MAF':
            PID_TAG = '132'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '108'
        elif LABEL == 'THROTTLE':
            PID_TAG = '91'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = '3236'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '3251'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = '4360'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = '84'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = '4766' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = '5054'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = '917' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = '3700' # 245

    elif (PROTOCOL == 'SAE_AVG'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_4766_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
    Time_vec = []
    Val_vec = []


    for i in range(len(data)):
        if "pids" in data[i]:
            if len(data[i]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[i]['pids'])):
                    State = data[i]['pids'][sub_pid_cnt]
                    if PID_TAG in State:
                        Time_vec.append(State[PID_TAG]['timestamp'])
                        Val_vec.append(State[PID_TAG]['value'][0])
      
                
    return Time_vec,Val_vec        
            

In [55]:
def regeneration_evidence_v3(vehicle_id, FLAG,
                            active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage, Median_Duration):
    
    
        if active_regeneration_start_time >= active_regeneration_end_time:
            duration_status = "Invalid Start and End Time"
            speed_status ="NA"
            return speed_status, duration_status, burn_quality_percentage

        # getting the active-regeneration duration
        active_regeneration_duration = (active_regeneration_end_time - active_regeneration_start_time)//(60*1000)

        print("this is the active-Regeneration Duration: ", active_regeneration_duration)

        # if the duration of active-regeneration is less than 10 minutes 
        if active_regeneration_duration <= 10:
                        duration_status = "insufficient"
                        speed_status ="NA"
                        return speed_status, duration_status, burn_quality_percentage
        
        # defining the intial and final time stamps needed for creating pre and post regeneration windows
        # We are selecting the time which is 1 hour before the start of the active-regeneration event
        # and 1 hour after the ative-regeneration event
        Start_TS = active_regeneration_start_time - 1*60*60*1000
        End_TS = active_regeneration_end_time + 1*60*60*1000

       
        
        # if FLAG is set to "US", use the US URL link
        if FLAG == 'US':
                
                RPM_RANGE = [800, 2000]
                SPEED_THRESHOLD = 80

                # loading the OBD_data
                URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
                
        # if FLAG is set to "IN", use the Indian URL link
        elif FLAG == 'IN':
                
                RPM_RANGE = [1000, 2000]
                SPEED_THRESHOLD = 40

                # loading the OBD_data
                URL = 'http://data-download.intangles.com:1883/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
                

        # getting the OBD data between start and end period
        r = requests.get(URL,stream=True)
        OBD_data = r.json()            
        
        # extracting the relevant parameter-ID
        # extracting the variable Differential Pressure
        Time, dp_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'DPFDP')
        # extracting the varible Engine RPM
        Time, rpm_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'ENGINE RPM')
        # extracting the variable SPEED
        Time, speed_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'SPEED')


        print(Time)
        
       

        # applying the RPM constraint
        # getting the correspnding values of different varaibles after applying constraint
        # Time1 constitutes of timestamps where the RPM conditions are met
        dp_rpm_constrained = []
        speed_rpm_constrained = []
        Time1 = []
        rpm_constrained = []

        # iterating through each timestamp to make the RPM constraint check
        for i in range(len(Time)):
                if rpm_inst_Value[i]>=RPM_RANGE[0] and rpm_inst_Value[i]<=RPM_RANGE[-1]:
                        dp_rpm_constrained.append(dp_inst_Value[i])
                        Time1.append(Time[i])
                        rpm_constrained.append(rpm_inst_Value[i])
                        speed_rpm_constrained.append(speed_inst_Value[i])
        
        # after applying the rpm-constraint check which are the nearest active regeneration start and end indices
        nearest_ar_start_idx = (np.abs(np.array(Time1) -  active_regeneration_start_time)).argmin()
        nearest_ar_end_idx = np.abs(np.array(Time1) -  active_regeneration_end_time).argmin()
        
        # if the nearest start and end indices comes out to be same (which implies after applying the rpm-constrint we are not left with any valid variable points)
        if (nearest_ar_start_idx == nearest_ar_end_idx):
                duration_status = "insufficient"
                speed_status ="NA"
                return speed_status, duration_status, burn_quality_percentage
                


        # calculating the mid-region of the active regeneration zone
        mid_idx = (nearest_ar_start_idx + nearest_ar_end_idx)//2
       
        # bringing both the pre and post window to same sizes
        pre_start_idx = 0
        post_end_idx = len(Time1)
        # if length of the pre-regeneration window > length of the post-regeneration window
        # bring the pre-window to same size as the post window
        if (mid_idx) > (len(Time1)- nearest_ar_end_idx):
            pre_start_idx = mid_idx - (len(Time1)- nearest_ar_end_idx)
        # if length of the pre-regeneration window < length of the post-regeneration window  
        # bring the post-window to same size as the pre window
        elif   (mid_idx) < (len(Time1)- nearest_ar_end_idx):
                post_end_idx = nearest_ar_end_idx + mid_idx  


        # fixing the pre_dp; pre_rpm and the pre_speed window
        pre_dp_window = dp_rpm_constrained[pre_start_idx:mid_idx]
        pre_rpm_window = rpm_constrained[pre_start_idx:mid_idx]
        pre_speed_window = speed_rpm_constrained[pre_start_idx:mid_idx]

        # fixing the post_dp; post_rpm and the post_speed window
        post_dp_window = dp_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_rpm_window = rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_speed_window = speed_rpm_constrained[nearest_ar_end_idx:post_end_idx]

        
        # quantifiying the burn_quality basis the burn_quality_percentage
        #   0 <= burn_quality_percentage < 0.33 --> low burn_quality
        if (burn_quality_percentage>=0 and burn_quality_percentage<0.33):
                burn_quality = 'low'
        #   0.33 <= burn_quality_percentage < 0.66 --> medium burn_quality        
        elif (burn_quality_percentage>=0.33 and burn_quality_percentage<0.66):
                burn_quality = 'medium' 
        #   0.66 <= burn_quality_percentage < 1 --> high burn_quality        
        elif (burn_quality_percentage>=0.66 and burn_quality_percentage<1):
                burn_quality = 'high' 
         #  burn_quality_percentage == 2 --> failed burn            
        elif burn_quality_percentage == 2:
                burn_quality = 'failed'

        # reconcilation logic for low and high burn-quality        
        if (burn_quality == 'low') or (burn_quality == 'high'):
                        
                        
                        # getting the partition for breaking the rpm-bins into two euqla bins 
                        bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
                        # getting the columns (we area capturing the count and mean of the dp values in ecah of the rpm-bins)
                        columns = [
                                'RPM Bin', 'Speed Bin', 'Count', 
                                'Mean'
                                ]

                        # capturing the decriptive statistics, and their corresponding rpm and speed bin
                        df_pre_statistics_all_vehicles = pd.DataFrame(columns = columns)
                        df_post_statistics_all_vehicles = pd.DataFrame(columns = columns)

                        pre_descriptive_statistics_all_bins = []
                        post_descriptive_statistics_all_bins = []
                        pre_rpm_bin_considered = []
                        post_rpm_bin_considered = []
                        pre_speed_bin_considered = []
                        post_speed_bin_considered = []

                        
                        print("this is the RPM Range: ", RPM_RANGE)

                        # getting the RPM_RANGE and creating bin-on those
                        for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                                # creating the low and high speed bins for each of the rpm-bins
                                # getting the pre-bins
                                pre_dp_in_rpm_low_speed_bin = []
                                pre_dp_in_rpm_high_speed_bin = []
                                # getting the post-bins
                                post_dp_in_rpm_low_speed_bin = []
                                post_dp_in_rpm_high_speed_bin = []


                                # parsing through all the enteries of the dp-window
                                for j in range(len(pre_dp_window)):  
                                        # if the rpm values are between the bin & (bin+bin_size)

                                        # checking the above condition for pre-rpm window
                                        if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                                # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                                        pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                                        
                                        # checking the above condition for the post-window
                                        if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                        # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                                        post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                                # if rpm with lower-speed bin
                                if len(pre_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_low_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        ]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                                        
                                if len(post_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_low_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                       ]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                                # if rpm with higher-speed bin                
                                if len(pre_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_high_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                                if len(post_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_high_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                                summary_statistics.loc['count'][0],
                                                                                summary_statistics.loc['mean'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))   
                                                                          
                                                
                        if len(pre_descriptive_statistics_all_bins):
                                        for j in range(len(pre_descriptive_statistics_all_bins)):  
                                                pre_row_data = []
                                                pre_row_data.append(pre_rpm_bin_considered[j]) #'RPM Bin'
                                                pre_row_data.append(pre_speed_bin_considered[j]) #'Speed Bin'
                                                pre_row_data.extend(pre_descriptive_statistics_all_bins[j]) 
                                                df_pre_statistics_all_vehicles.loc[len(df_pre_statistics_all_vehicles)] = pre_row_data
                                
                        if len(post_descriptive_statistics_all_bins):
                                        for j in range(len(post_descriptive_statistics_all_bins)):
                                                post_row_data = []  
                                                post_row_data.append(post_rpm_bin_considered[j]) #'RPM Bin'
                                                post_row_data.append(post_speed_bin_considered[j]) #'Speed Bin'
                                                post_row_data.extend(post_descriptive_statistics_all_bins[j]) 
                                                df_post_statistics_all_vehicles.loc[len(df_post_statistics_all_vehicles)] = post_row_data

                        # dropping off duplicates from pre and post dataframe
                        df_pre_statistics_all_vehicles = df_pre_statistics_all_vehicles.drop_duplicates()
                        df_post_statistics_all_vehicles = df_post_statistics_all_vehicles.drop_duplicates()

                        # renaming the columns for pre and post statistical dataframes
                        df_pre_statistics_all_vehicles.columns = ["Pre RPM Bin", "Pre Speed Bin", 
                                                                "Pre Count", "Pre Mean"]
                                
                        df_post_statistics_all_vehicles.columns = ["Post RPM Bin", "Post Speed Bin", 
                                                                "Post Count", "Post Mean"]
                                
                        # combining pre and post dataframes basis RPM and SPEED bin values 
                        df_statistics = pd.merge(df_pre_statistics_all_vehicles, df_post_statistics_all_vehicles, how='inner', 
                                                left_on=["Pre RPM Bin", "Pre Speed Bin"],
                                                right_on = ["Post RPM Bin", "Post Speed Bin"])
                                
                        # In case any duplicates generated while combining the two dataframes drop them
                        df_statistics = df_statistics.drop_duplicates()

                        # dropping all those statistical bins wherein the Pre or Post Count of values are less than 5
                        indices_remove = df_statistics[(df_statistics['Pre Count']<5) | (df_statistics['Post Count']<5)].index
                        df_statistics.drop(indices_remove, inplace=True)
                                
                        

                        print("Below given is the statistical DataFrame: ")

                        print(df_statistics)

                        fraction_good_bins = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]/df_statistics.shape[0]

                                
                        print("fraction of good bins: ", fraction_good_bins)
                        # for "high" soot burn
                        # the logic goes something like this
                        # if 0/4 bins has the post-dp < pre-dp, classify as low
                        # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'high' and fraction_good_bins == 0:
                                burn_quality = 'low'
                                burn_quality_percentage = burn_quality_percentage/3
                        elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage/2   

                        # for "low" soot burn
                        # the logic goes something like this
                        # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'low' and fraction_good_bins > 0.5:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage * 2
        high_speed_count = 0
        # giving the duration and speed status for ecah of the moderate and low burn_quality
        if burn_quality != 'high':
                # if 10 < active_regeneration_duration < Median Duration  --- > Active-Regeneration happened for sufficient duration
                if  active_regeneration_duration>10 and active_regeneration_duration<Median_Duration:
                        duration_status = 'moderate'
                elif  active_regeneration_duration>Median_Duration:
                        duration_status = 'sufficient'       
                # if for atleast for 50% of time; the vehicle is running at required speed for sufficient amount of time         
                for speed in speed_inst_Value:
                        if speed >= SPEED_THRESHOLD:
                                high_speed_count += 1
                if high_speed_count/len(speed_inst_Value) > 0.5:    
                        speed_status = 'sufficient'
                        
                elif  high_speed_count/len(speed_inst_Value) <= 0.5: 
                        speed_status = 'insufficient'
        # if burn_quality is high ; keep the speed and duration status as NA                
        elif burn_quality == 'high':
                speed_status = 'NA'  
                duration_status = 'NA' 

                           

        # return the speed, duration status and the time for which the function is executed
        return speed_status, duration_status, burn_quality_percentage              
                            

        

Playing Ground Starts

In [ ]:
start_time = time.time()

for i in range(2000):
    print(i)

end_time = time.time()

print("the start time is: ", start_time)
print("the end time is: ", end_time)
print("the time required to run the function is: ", (end_time-start_time))

In [24]:
# 5th May, 08:27 AM  to 5th May, 08:58 AM  'High'
# 3rd May, 08:12 AM to 3rd May, 08:25 AM 'High'
# 3rd May, 07:50 AM to 3rd May, 08:07 AM 'High'
# 3rd May, 07:47 AM to 3rd May, 07:49 AM 'Failed'
# 2nd May, 06:12 PM to 2nd May, 06:30 PM 'High'
# 28th Apr, 07:07 AM to 28th Apr, 07:30 AM  'High'

#AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
#AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714703400000, 1714654800000, 1714269600000]
#BURN_QUALITY = ['High', 'High', 'High', 'Failed', 'Low', 'High']
#BURN_QUALITY_PERCENTAGE = [0.98, 0.7, 0.85, 2, 0.2, 0.67]

AR_start_time = 1714877820000
AR_end_time = 1714879680000
BURN_QUALITY_PERCENTAGE = 0.2

In [59]:
vehicle_id = 1231762781519216640
FLAG = 'US'
active_regeneration_start_time = AR_start_time
active_regeneration_end_time = AR_end_time
burn_quality_percentage = 0.78
Median_Duration = 20

In [60]:
speed_status, duration_status, burn_quality_percentage = regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)

this is the active-Regeneration Duration:  31
[1714874318000.0, 1714874325667.0, 1714874333334.0, 1714874341001.0, 1714874348668.0, 1714874356335.0, 1714874364002.0, 1714874371669.0, 1714874379336.0, 1714874387003.0, 1714874394670.0, 1714874402337.0, 1714874418000.0, 1714874425667.0, 1714874433334.0, 1714874441001.0, 1714874448668.0, 1714874456335.0, 1714874464002.0, 1714874471669.0, 1714874479336.0, 1714874487003.0, 1714874494670.0, 1714874502337.0, 1714874518000.0, 1714874525667.0, 1714874533334.0, 1714874541001.0, 1714874548668.0, 1714874556335.0, 1714874564002.0, 1714874571669.0, 1714874579336.0, 1714874587003.0, 1714874594670.0, 1714874602337.0, 1714874618000.0, 1714874625667.0, 1714874633334.0, 1714874641001.0, 1714874648668.0, 1714874656335.0, 1714874664002.0, 1714874671669.0, 1714874679336.0, 1714874687003.0, 1714874694670.0, 1714874702337.0, 1714874718000.0, 1714874725667.0, 1714874733334.0, 1714874741001.0, 1714874748668.0, 1714874756335.0, 1714874764002.0, 1714874771669.0, 1

In [65]:
print(speed_status)
print(duration_status)
print(burn_quality_percentage)

NA
NA
0.78


In [20]:
time_interval_ls = []

ts = AR_start_time
for i in range(100):
    vehicle_id = 1231762781519216640
    FLAG = 'US'
    active_regeneration_start_time = AR_start_time + i*1000
    active_regeneration_end_time = AR_end_time + i*1000
    burn_quality_percentage = BURN_QUALITY_PERCENTAGE
    Median_Duration = 15
    speed_status, duration_status, time_interval= regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)
    time_interval_ls.append(time_interval)
    print(i, " iteration complete")
    print("tim taken in ", i , " iteration is: ", time_interval)

this is the active-Regeneration Duration:  31
[1714874318000.0, 1714874325667.0, 1714874333334.0, 1714874341001.0, 1714874348668.0, 1714874356335.0, 1714874364002.0, 1714874371669.0, 1714874379336.0, 1714874387003.0, 1714874394670.0, 1714874402337.0, 1714874418000.0, 1714874425667.0, 1714874433334.0, 1714874441001.0, 1714874448668.0, 1714874456335.0, 1714874464002.0, 1714874471669.0, 1714874479336.0, 1714874487003.0, 1714874494670.0, 1714874502337.0, 1714874518000.0, 1714874525667.0, 1714874533334.0, 1714874541001.0, 1714874548668.0, 1714874556335.0, 1714874564002.0, 1714874571669.0, 1714874579336.0, 1714874587003.0, 1714874594670.0, 1714874602337.0, 1714874618000.0, 1714874625667.0, 1714874633334.0, 1714874641001.0, 1714874648668.0, 1714874656335.0, 1714874664002.0, 1714874671669.0, 1714874679336.0, 1714874687003.0, 1714874694670.0, 1714874702337.0, 1714874718000.0, 1714874725667.0, 1714874733334.0, 1714874741001.0, 1714874748668.0, 1714874756335.0, 1714874764002.0, 1714874771669.0, 1

In [22]:
time_avg = 0
for time in time_interval_ls:
    time_avg += time

In [23]:
time_avg/len(time_interval_ls)

18.766044018268584

In [25]:
pd.DataFrame(time_interval_ls).describe()

,0
count,100.000000
mean,18.766044
std,19.948214
min,7.856055
25%,12.912015
50%,15.844348
75%,19.560974
max,208.207588


In [27]:
import plotly.express as px

# Create a boxplot using Plotly Express
fig = px.box(x=time_interval_ls)

# Show the plot
fig.show()

In [15]:
print(speed_status)
print(duration_status)

sufficient
sufficient


In [ ]:
print(len(pre_dp_window))
print(len(post_dp_window))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(pre_dp_window)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(post_dp_window)

In [ ]:
speed_status

In [ ]:
duration_status

In [ ]:
regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)

Playing Ground Ends

In [ ]:
speed_status, duration_status= regeneration_evidence_v3(vehicle_id, FLAG,
                                                        active_regeneration_start_time, active_regeneration_end_time, 
                                                        burn_quality_percentage, Median_Duration)

In [ ]:
def regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp):
        
        
        
        # getting the active-regeneration duration
        active_regeneration_duration = (end_timestamp - start_timestamp)//(60*1000)
       
        # if the duration of active-regeneration is less than 5 minutes 
        if active_regeneration_duration < 5:
                evidence = "Insufficient duration"
        # if number of samples for quantification are insuffiecient        
        elif burn_quality == 'NA':
                evidence = "Insufficient evidence"        
        # if burn-quality is either low or high        
        else:
                # if burn quality is either low or high
                if burn_quality == 'low' or burn_quality == 'high':
                        # getting the relevant bin-size & window-size  
                        bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
                        # getting the columns
                        columns = [
                                'RPM Bin', 'Speed Bin', 'Count', 
                                'Mean', 'Standard Deviation', 'Minimum',
                                '25th Percentile', '50th Percentile',
                                '75th Percentile', 'Maximum'
                                ]

                        # capturing the decriptive statistics, and their corresponsding rpm and speed bin
                        df_pre_statistics_all_vehicles = pd.DataFrame(columns = columns)
                        df_post_statistics_all_vehicles = pd.DataFrame(columns = columns)

                        pre_descriptive_statistics_all_bins = []
                        post_descriptive_statistics_all_bins = []
                        pre_rpm_bin_considered = []
                        post_rpm_bin_considered = []
                        pre_speed_bin_considered = []
                        post_speed_bin_considered = []


                        # getting the RPM_RANGE and creating bin-on those
                        for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                                # creating the low and high speed bins for each of the rpm-bins
                                # getting the pre-bins
                                pre_dp_in_rpm_low_speed_bin = []
                                pre_dp_in_rpm_high_speed_bin = []
                                # getting the post-bins
                                post_dp_in_rpm_low_speed_bin = []
                                post_dp_in_rpm_high_speed_bin = []


                                # parsing through all the enteries of the dp-window
                                for j in range(len(pre_dp_window)):  
                                        # if the rpm values are between the bin & (bin+bin_size)

                                        # checking the above condition for pre-rpm window
                                        if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                                # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                                        pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                    pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                                        
                                        # checking the above condition for the post-window
                                        if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                        # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                                        post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                                # if rpm with lower-speed bin
                                if len(pre_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_low_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                        
                                if len(post_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_low_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                                # if rpm with higher-speed bin                
                                if len(pre_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_high_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0],
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                                if len(post_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_high_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                                summary_statistics.loc['count'][0],
                                                                                summary_statistics.loc['mean'][0],
                                                                                summary_statistics.loc['std'][0], 
                                                                                summary_statistics.loc['min'][0], 
                                                                                summary_statistics.loc['25%'][0],
                                                                                summary_statistics.loc['50%'][0], 
                                                                                summary_statistics.loc['75%'][0],
                                                                                summary_statistics.loc['max'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))        
                                                
                                        
                        if len(pre_descriptive_statistics_all_bins):
                                        for j in range(len(pre_descriptive_statistics_all_bins)):  
                                                pre_row_data = []
                                                pre_row_data.append(pre_rpm_bin_considered[j]) #'RPM Bin'
                                                pre_row_data.append(pre_speed_bin_considered[j]) #'Speed Bin'
                                                pre_row_data.extend(pre_descriptive_statistics_all_bins[j]) 
                                        
                                                df_pre_statistics_all_vehicles.loc[len(df_pre_statistics_all_vehicles)] = pre_row_data
                                
                        if len(post_descriptive_statistics_all_bins):
                                        for j in range(len(post_descriptive_statistics_all_bins)):
                                                post_row_data = []  
                                                post_row_data.append(post_rpm_bin_considered[j]) #'RPM Bin'
                                                post_row_data.append(post_speed_bin_considered[j]) #'Speed Bin'
                                                post_row_data.extend(post_descriptive_statistics_all_bins[j]) 
                                                df_post_statistics_all_vehicles.loc[len(df_post_statistics_all_vehicles)] = post_row_data

                        df_pre_statistics_all_vehicles = df_pre_statistics_all_vehicles.drop_duplicates()
                        df_post_statistics_all_vehicles = df_post_statistics_all_vehicles.drop_duplicates()


                        print(df_pre_statistics_all_vehicles)



                        df_pre_statistics_all_vehicles.columns = ["Pre RPM Bin", "Pre Speed Bin", 
                                                                "Pre Count", "Pre Mean", 
                                                                "Pre Standard Deviation", "Pre Minimum",
                                                                "Pre 25th Percentile", "Pre 50th Percentile",
                                                                "Pre 75th Percentile", "Pre Maximum"]
                        
                        df_post_statistics_all_vehicles.columns = ["Post RPM Bin", "Post Speed Bin", 
                                                                "Post Count", "Post Mean", 
                                                                "Post Standard Deviation", "Post Minimum",
                                                                "Post 25th Percentile", "Post 50th Percentile",
                                                                "Post 75th Percentile", "Post Maximum"]
                        
                        df_statistics = pd.merge(df_pre_statistics_all_vehicles, df_post_statistics_all_vehicles, how='inner', 
                                                left_on=["Pre RPM Bin", "Pre Speed Bin"],
                                                right_on = ["Post RPM Bin", "Post Speed Bin"])
                        
                        df_statistics = df_statistics.drop_duplicates()

                        indices_remove = df_statistics[(df_statistics['Pre Count']<5)].index
                        df_statistics.drop(indices_remove, inplace=True)
                        
                        
                        #df_statistics.loc[(df_statistics['Pre Count']<5), ['Pre Mean', 'Post Mean']] = 0 

                        df_statistics["Status"] = "Good"
                        df_statistics.loc[(df_statistics["Post Mean"]>df_statistics["Pre Mean"]), "Status"] = "Bad"
                        
                        c = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]

                        fraction_good_bins = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]/df_statistics.shape[0]

                        
                        print("fraction of good bins: ", fraction_good_bins)
                        # for "high" soot burn
                        # the logic goes something like this
                        # if 0/4 bins has the post-dp < pre-dp, classify as low
                        # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'high' and fraction_good_bins == 0:
                                burn_quality = 'low'
                                burn_quality_percentage = burn_quality_percentage/3
                        elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage/2   

                         # for "low" soot burn
                        # the logic goes something like this
                        # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'low' and fraction_good_bins > 0.5:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage * 2
                                  
                        if burn_quality == 'low':
                                print(df_statistics)
                                if df_statistics.loc[(df_statistics["Pre Speed Bin"] == str(">")+str(SPEED_THRESHOLD)),"Pre Count"].sum() == 0:
                                                evidence = "Speed conditions were insufficiently met"
                                else:
                                        status_ls = df_statistics.loc[(df_statistics["Pre Speed Bin"] == str(">")+str(SPEED_THRESHOLD)),"Status"].tolist()
                                        any_bad = any(element == "bad" for element in status_ls)
                                        if any_bad:
                                                evidence = "Speed conditions were insufficiently met"
                                        else:
                                                evidence =  "Higher soot load despite speed conditions were met"
                        if burn_quality == 'high':  
                                evidence = "Duration and speed conditions were met"
                elif burn_quality == 'medium' :
                        evidence = "Duration and Speed conditions were moderately met"

        return evidence, burn_quality                        


                
        
        
        


        

In [ ]:
def resample_PID_data(X_Value,Number_of_features):

    # DPFDP has minimum length DATA[1,:], so resample all other variables  w.r.t. DPFDP
    Samples = len(X_Value[1])
    #print(len(X_Value[0]))
    DATA = np.zeros((Number_of_features,Samples))
    TEMP = np.transpose(np.array(X_Value[1]))
 
    for cnt in range(0,Number_of_features):
        if (len(X_Value[cnt]) > 0):
            DATA[cnt,:] = np.transpose(np.array(signal.resample(X_Value[cnt], Samples)))

    DATA[1,:] = TEMP

    return DATA

In [ ]:
def extract_PID_data_v3(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = '106'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '92'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '190'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '183'
        elif LABEL == 'MAF':
            PID_TAG = '132'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '108'
        elif LABEL == 'THROTTLE':
            PID_TAG = '91'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = '3236'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '3251'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = '4360'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = '84'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = '4766' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = '5054'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = '917' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = '3700' # 245

    elif (PROTOCOL == 'SAE_AVG'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_4766_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
    Time_vec = []
    Val_vec = []


    for i in range(len(data)):
        if "pids" in data[i]:
            if len(data[i]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[i]['pids'])):
                    State = data[i]['pids'][sub_pid_cnt]
                    if PID_TAG in State:
                        Time_vec.append(State[PID_TAG]['timestamp'])
                        Val_vec.append(State[PID_TAG]['value'][0])
      
                
    return Time_vec,Val_vec        
            

In [ ]:
PATH = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_DP_included/"
VARIABLES = ['ENGINE RPM', 'DPFDP', 'SPEED']

# dir_list = os.listdir(PATH)
# print(dir_list)
# DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
#for data_packet_cnt in range(0,len(dir_list)):
#OBD_data_path = PATH + dir_list[data_packet_cnt]
path = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640/" 
#OBD_data_path = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_1709231400000_1714933800000/5TH_MY_REGEN/1231762781519216640_1714897426000_1714933800000.json"
#OBD_data = [json.loads(line) for line in open(OBD_data_path, 'r')]

In [ ]:
vehicle = 1231762781519216640
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
for i in range(len(AR_start_time)):
    print(AR_start_time[i])
    print(AR_end_time[i])
    Start_TS = AR_start_time[i] - 1*60*60*1000
    End_TS = AR_end_time[i] + 1*60*60*1000
    print("this is the starting timestamp ", Start_TS)
    print("this is the ending timestamp ", End_TS)
    URL = ' https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle) +  "/" + str(Start_TS) + "/" + str(End_TS)
    r = requests.get(URL,stream=True)
    OBD_data = r.json() 
    print(OBD_data[0])
    dp_inst_Time, dp_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'DPFDP')
    rpm_inst_Time, rpm_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'ENGINE RPM')
    throttle_inst_Time, throttle_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'THROTTLE')
    break

In [ ]:
dp_inst_Time[-1]

In [ ]:
dp_rpm_constrained = []
throttle_rpm_constrained = []
time_rpm_constrained = []
rpm_constrained = []
for i in range(len(dp_inst_Time)):
    if rpm_inst_Value[i]>=1000 and rpm_inst_Value[i]<=2000:
        dp_rpm_constrained.append(dp_inst_Value[i])
        time_rpm_constrained.append(dp_inst_Time[i])
        rpm_constrained.append(rpm_inst_Value[i])
        throttle_rpm_constrained.append(throttle_inst_Value[i])


In [ ]:
#plt.hist(throttle_rpm_constrained)
higher_throttle = []
for i in range(len(throttle_rpm_constrained)):
    if throttle_rpm_constrained[i]>=30:
        higher_throttle.append(throttle_rpm_constrained[i])
 


In [ ]:
len(higher_throttle)/len(throttle_rpm_constrained)

In [ ]:
dp_rpm_throttle_constrained = []
time_rpm_throttle_constrained = []
rpm_throttle_constrained = []
for i in range(len(dp_rpm_constrained)):
    if throttle_inst_Value[i]>=50:
        dp_rpm_throttle_constrained.append(dp_inst_Value[i])
        time_rpm_throttle_constrained.append(dp_inst_Time[i])
        rpm_throttle_constrained.append(rpm_inst_Value[i])


In [ ]:
_, start_idx = find_nearest(time_rpm_throttle_constrained, AR_start_time[0])
_, end_idx = find_nearest(time_rpm_throttle_constrained, AR_end_time[0])
_, last_idx = find_nearest(time_rpm_throttle_constrained, AR_end_time[0] + 60*60*1000)
mid_idx = (end_idx + start_idx)//2
print(mid_idx - start_idx + 1)
print(last_idx-end_idx+1)

In [ ]:
plt.hist(rpm_throttle_constrained)

In [ ]:
plt.hist(rpm_constrained)

In [ ]:
pre_rpm = rpm_throttle_constrained[:mid_idx]
post_rpm = rpm_throttle_constrained[end_idx:]

In [ ]:
# dp_rpm_throttle_constrained.append(dp_inst_Value[i])
# time_rpm_throttle_constrained.append(dp_inst_Time[i])
# rpm_throttle_constrained.append(rpm_inst_Value[i])


In [ ]:
pre_low_rpm = []
post_low_rpm = []
pre_high_rpm = []
post_high_rpm = []
for i in range(len(pre_rpm)):
    if pre_rpm[i]>=1000 and pre_rpm[i]<1400:
        pre_low_rpm.append(pre_rpm[i])
    elif pre_rpm[i]>=1400 and pre_rpm[i]<=2000:
        pre_high_rpm.append(pre_rpm[i])
for i in range(len(post_rpm)):
    if post_rpm[i]>=1000 and post_rpm[i]<1400:
        post_low_rpm.append(post_rpm[i])
    elif post_rpm[i]>=1400 and post_rpm[i]<=2000:
        post_high_rpm.append(post_rpm[i])        



In [ ]:
print(len(pre_low_rpm))
print(len(pre_high_rpm))
print(len(post_low_rpm))
print(len(post_high_rpm))

Checking of the logic done above

In [ ]:
Time = []
rpm_Val = []
speed_Val = []
dp_Val= []

dir_packets = os.listdir(path)
dir_packets.sort()
for packet in dir_packets:
    dir_path = path + str(packet)
    OBD_data = [json.loads(line) for line in open(dir_path, 'r')]
    if len(OBD_data[0]['results']['data']):
        rpm_inst_Time, rpm_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'ENGINE RPM')
        dp_inst_Time, dp_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'DPFDP')
        speed_inst_Time, speed_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'SPEED')
        Time.extend(rpm_inst_Time)
        rpm_Val.extend(rpm_inst_Value)
        speed_Val.extend(speed_inst_Value)
        dp_Val.extend(dp_inst_Value)
        




In [ ]:
import plotly.graph_objects as go

# Create some data
x = np.arange(len(dp_Val))
y = dp_Val

# Create a Plotly figure
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', marker=dict(color='blue')))

# Update layout
fig.update_layout(title='DP-Value vs Engine-Hours', xaxis_title='Engine-Hours', yaxis_title='DP Value')

# Show the figure
fig.show()

In [ ]:
# applying RPM constraint on the DP-Values
dp_value_rpm_constrained = []
rpm_value_constrained = []
speed_value_rpm_constrained = []
time_constrained = []
for i in range(len(dp_Val)):
    if rpm_Val[i]>=1000 and rpm_Val[i]<=2000:
        rpm_value_constrained.append(rpm_Val[i])
        dp_value_rpm_constrained.append(dp_Val[i])
        speed_value_rpm_constrained.append(speed_Val[i])
        time_constrained.append(Time[i])





In [ ]:
import plotly.graph_objects as go

# Create some data
x = np.arange(len(dp_value_rpm_constrained))
y = dp_value_rpm_constrained

# Create a Plotly figure
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', marker=dict(color='blue')))

# Update layout
fig.update_layout(title='DP-Value Constrained vs Engine-Hours', xaxis_title='Engine-Hours', yaxis_title='DP Value')

# Show the figure
fig.show()

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000
col=[]
for i in range(0,len(dp_value_rpm_constrained)):
    if 1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500:
        col.append('[(1000, 1500)] or [lowrpm]')   
    elif rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000:
        col.append('[(1500, 2000)] or [highrpm]')
data = {'Engine_Hours': np.arange(len(dp_value_rpm_constrained)), 'DP_Value': dp_value_rpm_constrained}
df = pd.DataFrame(data)
fig =px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500)] or [lowrpm]':'blue',
                                                                                    '[(1500, 2000)] or [highrpm]': 'orange',
                                                                                       }) 
fig.show()                                                                                           
   

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000 and 80 are RPM and speed thresholds respectively
import plotly.express as px
col=[]
for i in range(0,len(dp_value_rpm_constrained)):
    if (1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500) and speed_value_rpm_constrained[i]<=80:
        col.append('[(1000, 1500), <=80] or [lowrpm, lowspeed]')
    elif (1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500) and speed_value_rpm_constrained[i]>80:
        col.append('[(1000, 1500), >80] or [lowrpm, lowspeed]')       
    elif (rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000) and speed_value_rpm_constrained[i]<=80:
        col.append('[(1500, 2000), <=80] or [highrpm, lowspeed]')
    elif (rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000) and speed_value_rpm_constrained[i]>80:
        col.append('[(1500, 2000), >80] or [highrpm, lowspeed]')    
data = {'Engine_Hours': np.arange(len(dp_value_rpm_constrained)), 'DP_Value': dp_value_rpm_constrained}
df = pd.DataFrame(data)
fig =px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500), <=80] or [lowrpm, lowspeed]':'blue',
                                                                                    '[(1000, 1500), >80] or [lowrpm, lowspeed]': 'green',
                                                                                    '[(1500, 2000), <=80] or [highrpm, lowspeed]': 'orange',
                                                                                    '[(1500, 2000), >80] or [highrpm, lowspeed]': 'purple'
                                                                                       }) 
fig.show()    

In [ ]:
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
BURN_QUALITY = ['high', 'high', 'high', 'low', 'high', 'high']
for i in range(len(AR_start_time)):
       print(BURN_QUALITY[i])
       _, start_idx = find_nearest(time_constrained, AR_start_time[i])
       _, end_idx = find_nearest(time_constrained, AR_end_time[i])
       mid_idx = (start_idx + end_idx)//2
       pre_dp_window = dp_value_rpm_constrained[mid_idx-50:mid_idx]
       post_dp_window = dp_value_rpm_constrained[end_idx: end_idx+50]

       pre_rpm_window = rpm_value_constrained[mid_idx-50:mid_idx]
       post_rpm_window = rpm_value_constrained[end_idx: end_idx+50]

       pre_speed_window = speed_value_rpm_constrained[mid_idx-50:mid_idx]
       post_speed_window = speed_value_rpm_constrained[end_idx: end_idx+50]

       SPEED_THRESHOLD = 80
       RPM_RANGE = [1000, 2000]
       burn_quality = BURN_QUALITY[i]
       burn_quality_percentage = 0.8
       start_timestamp = AR_start_time[i]
       end_timestamp = AR_end_time[i]
       
       evidence, burn_quality = regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp)
       print("this is the evidence: ", evidence)                   
       print(i)                   
       #break                   

In [ ]:
evidence

In [ ]:
df_statistics, evidence = regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp)

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000
col=[]
        for i in range(0,len(X_2[1,:])):
            if 1000<=X_2[0][i] and X_2[0][i]<1500 and X_2[5][i]>40:
                col.append('[(1000, 1500) & >40] or [lowrpm, highspeed]')
            elif 1000<=X_2[0][i] and X_2[0][i]<1500 and X_2[5][i]<=40:
                col.append('[(1000, 1500) & <=40] or [lowrpm, lowspeed]')    
            elif X_2[0][i]>=1500 and X_2[0][i]<2000 and X_2[5][i]>40:
                col.append('[(1500, 2000) & >40] or [highrpm, highspeed]')
            elif X_2[0][i]>=1500 and X_2[0][i]<2000 and X_2[5][i]<=40:
                col.append('[(1500, 2000) & <=40] or [highrpm, lowspeed]')   
                

        data = {'Engine_Hours': np.arange(len(T_2)), 'DP_Value': X_2[1,:]}
        df = pd.DataFrame(data)    
        #utc_Time = [datetime.utcfromtimestamp(ms / 1000).strftime('%Y-%m-%d %H:%M:%S') for ms in T_2] 

        # fig = px.scatter(df, x='utc_Time', y='DP_Value', color=col, color_discrete_map={'(1000, 1500)': 'green', '(1500, 2000)': 'yellow'})
        # fig.add_trace(go.Scatter(x=np.arange(len(X_4[1])), y=regeneration_binary_signal, mode='markers', name='Regeneration Signal', marker=dict(color='red')))
        fig = px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500) & >40] or [lowrpm, highspeed]':'green',
                                                                                        '[(1000, 1500) & <=40] or [lowrpm, lowspeed]':'blue',
                                                                                        '[(1500, 2000) & >40] or [highrpm, highspeed]': 'orange',
                                                                                        '[(1500, 2000) & <=40] or [highrpm, lowspeed]': 'purple'})
       fig.update_layout(
            title={
                'text': str(df_vector_to_be_explored1.loc[(df_vector_to_be_explored1["Vehicle_ID"]==int(vehicle)) & 
                                                                 (df_vector_to_be_explored1["Active Regeneration Start Time"]==start_regeneration_time[start_idx])
                                                                 &  (df_vector_to_be_explored1["Active Regeneration End Time"]==end_regeneration_time[start_idx]),
                                                                 'burn_quantification' ].values[0]) + " soot" + " burn"
                                                                 ,
                'y': 0.9,  # Adjust the vertical position
                'x': 0.5,  # Adjust the horizontal position
                'xanchor': 'center',
                'yanchor': 'top',
                'font': dict(
                    family="Arial",
                    size=24,
                    color="black"
                )
            }
        )

        fig.show()
        

In [ ]:
# 5th May, 08:27 AM  to 5th May, 08:58 AM  'High'
# 3rd May, 08:12 AM to 3rd May, 08:25 AM 'High'
# 3rd May, 07:50 AM to 3rd May, 08:07 AM 'High'
# 3rd May, 07:47 AM to 3rd May, 07:49 AM 'Failed'
# 2nd May, 06:12 PM to 2nd May, 06:30 PM 'High'
# 28th Apr, 07:07 AM to 28th Apr, 07:30 AM  'High'
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
BURN_QUALITY = ['High', 'High', 'High', 'Failed', 'High', 'High']

In [ ]:
for i in range(len(AR_start_time)):
    regeneration_start_idx = Time.index(AR_start_time[i])
    regeneration_end_idx = Time.index(AR_end_time[i])
    mid_idx = (regeneration_start_idx + regeneration_end_idx)//2
    pre_dp_window = dp_Val[(mid_idx - 50):mid_idx]
    pre_rpm_window 

In [ ]:
OBD_data[0]['results']

In [ ]:
VARIABLES = ['ENGINE RPM', 'DPFDP', 'SPEED']
Time_vec,Val_vec = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE', 'DPFDP')
rpm_Time = []
rpm_Val = []

dp_Time = []
dp_Val = []

speed_Time = []
speed_Val = []
for var_type in VARIABLES:
    X_Time, X_Value = extract_PID_data_v3(OBD_data[0]['results']['data'],'SAE_AVG',var_type)
    if var_type == 'ENGINE RPM':
        rpm_Time.extend(X_Time)
        rpm_Val.extend(X_Value)
    elif var_type == 'DPFDP':
        dp_Time.extend(X_Time)
        dp_Val.extend(X_Value)  
    elif var_type == 'SPEED':
        speed_Time.extend(X_Time)
        speed_Val.extend(X_Value)        

In [ ]:
print("rpm frequency")
ls_rpm_frequency = []
for i in range(1, len(rpm_Time)):
    ls_rpm_frequency.append((rpm_Time[i]-rpm_Time[i-1])/(1000))


In [ ]:
print("dp frequency")
ls_dp_frequency = []
for i in range(1, len(dp_Time)):
    ls_dp_frequency.append((dp_Time[i]-dp_Time[i-1])/(1000))

In [ ]:
print("speed frequency")
ls_speed_frequency = []
for i in range(1, len(dp_Time)):
    ls_speed_frequency.append((dp_Time[i]-dp_Time[i-1])/(1000))

In [ ]:
active_regeneration_start = 1714877820000
active_regeneration_end = 1714879680000

_, active_regeneration_start_idx =  find_nearest(dp_Time, active_regeneration_start)
_, active_regeneration_end_idx =  find_nearest(dp_Time, active_regeneration_end)


In [ ]:
mid_idx = (active_regeneration_start_idx + active_regeneration_end_idx)//2

In [ ]:
mid_idx

In [ ]:
# pre_dp_window, post_dp_window, 
# pre_rpm_window, post_rpm_window,
# pre_speed_window, post_speed_window,
# SPEED_THRESHOLD, RPM_RANGE, burn_quality,
# burn_quality_percentage,
# start_timestamp, end_timestamp
pre_dp_window = dp_Val[(mid_idx - 50): mid_idx]
pre_rpm_window = rpm_Val[(mid_idx - 50): mid_idx]
pre_speed_window = speed_Val[(mid_idx - 50): mid_idx]


In [ ]:
pre_dp_window

Ignore

In [ ]:
def resample_PID_data(X_Value,Number_of_features):

    # DPFDP has minimum length DATA[1,:], so resample all other variables  w.r.t. DPFDP
    Samples = len(X_Value[1])
    #print(len(X_Value[0]))
    DATA = np.zeros((Number_of_features,Samples))
    TEMP = np.transpose(np.array(X_Value[1]))
 
    for cnt in range(0,Number_of_features):
        if (len(X_Value[cnt]) > 0):
            DATA[cnt,:] = np.transpose(np.array(signal.resample(X_Value[cnt], Samples)))

    DATA[1,:] = TEMP

    return DATA

In [ ]:
PATH = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_1709231400000_1714933800000/5TH_MY_REGEN/"

In [ ]:
dir_list = os.listdir(PATH)
print(dir_list)
DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
for data_packet_cnt in range(0,len(dir_list)):
#for data_packet_cnt in range(0,5):
    OBD_data_path = PATH + dir_list[data_packet_cnt] 
    OBD_data = [json.loads(line) for line in open(OBD_data_path, 'r')]

In [ ]:
#VARIABLES = ['ENGINE RPM', 'DPFDP']
VARIABLES = ['ENGINE RPM']

In [ ]:
DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
T_L = []
V_L = []
T1 = []
for var_type in VARIABLES:
    X_Time, X_Value = extract_PID_data_v3(OBD_data[0]['results']['data'],'SAE',var_type)
    T_L.append(np.array((X_Time), dtype=np.int64))
    V_L.append(np.array((X_Value), dtype=float)) 



# if (len(V_L[0]) > 0):
#     Temp_time = np.array(T_L[1]) # its DP time so no need to resample
#     Resample_data = resample_PID_data(V_L,len(VARIABLES)) # cross check by plot
    
#     DATA = np.concatenate((DATA,Resample_data),axis=1)
#     T1 = np.concatenate((T1,Temp_time),axis=0)

In [ ]:
T1.sort()

In [ ]:
X_Time[0]

In [ ]:
X_Time[-1]

In [2]:
from datetime import datetime, timezone

def convert_ms_to_utc(ms):
    # Convert milliseconds to seconds
    seconds = ms / 1000.0
    # Create a datetime object from the seconds, with UTC timezone
    utc_time = datetime.utcfromtimestamp(seconds).replace(tzinfo=timezone.utc)
    return utc_time

milliseconds = 1621416523000  # Example milliseconds

utc_time = convert_ms_to_utc(milliseconds)
print("UTC time:", utc_time)


UTC time: 2021-05-19 09:28:43+00:00


In [15]:
from datetime import datetime, timezone

def convert_utc_to_ms(utc_time):
    # Convert UTC time to POSIX timestamp
    posix_timestamp = utc_time.timestamp()
    # Convert POSIX timestamp to milliseconds
    milliseconds = int(posix_timestamp * 1000)
    return milliseconds

# Example UTC time
utc_time = datetime.utcnow().replace(tzinfo=timezone.utc)
utc_time = datetime.fromisoformat('2024-05-17 16:56:30+00:00')


milliseconds = convert_utc_to_ms(utc_time)
print("Milliseconds since the epoch:", milliseconds)


Milliseconds since the epoch: 1715964960000
